In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm


In [ ]:
class CipsTest:
  '''
  Implementation of the standard Cross-Sectionally Augmented Dickey-Fuller 
  precedure to test for non-stationarity I(1) in panel data.
  -------
  _____________\n
  PARAMETERS:
  ----------
  ------
  - *df*: A standart Pandas DataFrame containing panelized data. \n
    Ensure that the DataFrame contains the following columns in this exact order: \n
    0 - a column of spatial units. Must contain homogenous data, e.g. only countries, companies, regions, etc. \n
    1 - temporal column. Must contain homogenous data, e.g. only years, months, quarters, etc. \n
    2 - target variable. Must not contain NaN Values. An Error will be raised otherwise. \n
  
  - *T*: Your Temporal window. Will be used to determine the test critical value.
  - *N*: Your Spatial window. Will be used to determine the test critical value.
  
  - *trend*: State whether your ADF model has a trend or not. Will be used to determine the test critical value.
  
  - *intercept*: State whether ADF model has an intercept or not. Will be used to determine the test critical value.
  
  - *n_lags*: Determine the amount of lags in the Augmented Dickey-Fuller regression.
    For relatively small T going over 3 lags may lead to the test losing power.
    
  - *level*: Value of significance to conduct the test at (in %%). Only 5 and 1% are allowed.
  '''
  def __init__(self, df: pd.DataFrame, T: int, N: int, trend: bool =  False, intercept: bool = False, n_lags: int = 2, level: int = 5) -> None:
    self.__df = df
    self.__T = T
    self.__N = N
    self.__trend = trend
    self.__C = intercept
    self.__n_lags = n_lags
    self.__alpha = level/100
    self.__df.rename(columns={self.__df.columns[0]:'SpUnit', self.__df.columns[1]:'time', self.__df.columns[2]:'target'})
    self.verify()
    CipsTest.__build_tables()
  
  def verify(self) -> None:
    if self.__df.target.isnull().sum() > 0:
      raise TypeError('Values must NOT be NaN!')
      self.__del__()
    if self.__alpha != 0.01 and self.__alpha != 0.05:
      raise ValueError('Significance level must be either 1 or 5!')
      self.__del__()
      
  @classmethod
  def __build_tables(cls) -> None:
    cls.NTNC_1P = pd.read_excel('CADF_Crit_Values.xlsx', sheet_name='NTNC_1P')
    cls.NTNC_5P = pd.read_excel('CADF_Crit_Values.xlsx', sheet_name='NTNC_5P')
    cls.NTC_1P = pd.read_excel('CADF_Crit_Values.xlsx', sheet_name='NTC_1P')
    cls.NTC_5P = pd.read_excel('CADF_Crit_Values.xlsx', sheet_name='NTC_5P')
    cls.TC_1P = pd.read_excel('CADF_Crit_Values.xlsx', sheet_name='TC_1P')
    cls.TC_5P = pd.read_excel('CADF_Crit_Values.xlsx', sheet_name='TC_5P')
    
  def __del__(self) -> None:
    pass